In [35]:
import hnswlib
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [36]:
datasetImages, datasetLabels = readMnist()

datasetImages :  (70000, 784)
datasetLabels :  (70000,)


***Create HNSW Index***

In [37]:
def createIndex(indexMethod, datasetImages):
    d = datasetImages.shape[1] 
    num_elements = datasetImages.shape[0] 
    time_start = perf_counter()
    index = indexMethod(space = 'l2', dim = d)
    index.init_index(max_elements = num_elements, ef_construction = 200, M = 16)
    ids = np.arange(num_elements)
    index.add_items(datasetImages, ids)
    index.set_ef(120)
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    print(f'Took {totalTime:.3f} seconds')
    return (index, totalTime)
(indexedStruct, time) = createIndex(hnswlib.Index, datasetImages)

Took 25.589 seconds


In [38]:
print(indexedStruct)

<hnswlib.Index(space='l2', dim=784)>


In [ ]:
# (min, max) = createIndexNumerous(createIndex, NearestNeighbors, datasetImages, 10)
# print('min : ', min, '\n','max : ', max,)

In [39]:
indexes = []
distances = []
def measureTime(par, indexes, distances):
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        index, distance = indexedStruct.knn_query(xq, k = 100)
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        indexes.append(index[0])
        distances.append(np.sqrt(distance[0]))
    # report the duration
    print(f'Took {totalTime:.3f} seconds')
measureTime(1000, indexes, distances)

Took 0.750 seconds


In [40]:
# (min, max) = measureTimeNumerous(measureTime, 10)
# print('min : ', min, '\n', 'max : ', max,)

In [41]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [42]:
print('indexes : ', indexes.shape,'\n', 'distances : ', distances.shape,)

indexes :  (1000, 100) 
 distances :  (1000, 100)


In [43]:
(trueIndexes, trueDistances) = readDB()

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [44]:
compareFirstTen(indexes, distances, trueIndexes, trueDistances)

0 || 0
0.0 || 0.0
32248 || 32248
1561.4724 || 1561.4724
8728 || 8728
1591.6014 || 1591.6014
18932 || 18932
1594.7191 || 1594.7191
30483 || 30483
1596.7095 || 1596.7095
63470 || 63470
1600.8738 || 1600.8738
65325 || 65325
1603.479 || 1603.479
24149 || 24149
1604.4469 || 1604.4469
42338 || 42338
1604.8876 || 1604.8876
52295 || 52295
1605.7092 || 1605.7092


In [45]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)

0.9037
1.0000
1.0000
